In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gmplot 
import statistics

In [2]:
Tashu = pd.read_csv('./Tashu/tashu.csv')
Tashu = Tashu.dropna()
Tashu['Count'] = 1
Tashu = Tashu[Tashu['RENT_STATION'] != Tashu['RETURN_STATION']]
Tashu.head(5)

C:\Users\KIMJINSEOP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RENT_STATION,RENT_DATE,RETURN_STATION,RETURN_DATE,Count
0,43.0,20130101055603,34.0,2.01301e+13,1
2,2.0,20130101060406,10.0,2.01301e+13,1
3,106.0,20130101105305,105.0,2.01301e+13,1
5,21.0,20130101113953,105.0,2.01301e+13,1
6,90.0,20130101120833,91.0,2.01301e+13,1


In [3]:
RentData = Tashu.groupby(['RENT_STATION', 'RETURN_STATION'])['Count'].sum().nlargest(10).reset_index()
RentData['RENT_STATION'] = RentData['RENT_STATION'].astype(int)
RentData['RETURN_STATION'] = RentData['RETURN_STATION'].astype(int)
RentData.head(10)

,RENT_STATION,RETURN_STATION,Count
0,21,105,17220
1,105,21,12154
2,56,32,11868
3,32,56,11118
4,105,22,8074
5,107,105,7912
6,22,105,7463
7,21,22,6556
8,1,3,6135
9,105,107,6065


In [4]:
station = pd.read_csv('./Tashu/Station.csv')
del station['키오스크번호']
del station['구별']
del station['명칭']
del station['위치']
del station['주소']
del station['거치대']
station = station.rename(columns = {'번호': 'RENT_STATION'})
station.head()

,RENT_STATION,좌표
0,1,"36.374325,127.387462"
1,2,"36.374472,127.392241"
2,3,"36.369855,127.388749"
3,4,"36.368192,127.379281"
4,5,"36.365034,127.389361"


In [5]:
RentData = pd.merge(RentData,station, on = ['RENT_STATION'], how = 'left')
station = station.rename(columns = {'RENT_STATION': 'RETURN_STATION'})
RentData = RentData.rename(columns = {'좌표': 'RENT_LOC'})
RentData = pd.merge(RentData,station, on = ['RETURN_STATION'], how = 'left')
RentData = RentData.rename(columns = {'좌표': 'RETURN_LOC'})
RentData.head()

,RENT_STATION,RETURN_STATION,Count,RENT_LOC,RETURN_LOC
0,21,105,17220,"36.373457,127.359293","36.364177,127.358845"
1,105,21,12154,"36.364177,127.358845","36.373457,127.359293"
2,56,32,11868,"36.361736, 127.344994","36.359293, 127.354503"
3,32,56,11118,"36.359293, 127.354503","36.361736, 127.344994"
4,105,22,8074,"36.364177,127.358845","36.370263,127.362136"


In [6]:
import folium
top10_lat= RentData['RENT_LOC'].apply(lambda e: e.split(",")[0]).astype(float)
top10_lon= RentData['RENT_LOC'].apply(lambda e: e.split(",")[1]).astype(float)
rtop10_lat= RentData['RETURN_LOC'].apply(lambda e: e.split(",")[0]).astype(float)
rtop10_lon= RentData['RETURN_LOC'].apply(lambda e: e.split(",")[1]).astype(float)
map_Top10 = folium.Map(location = [statistics.median(top10_lat),statistics.median(top10_lon)], zoom_start = 14)

index = 0
for i in top10_lat :
    folium.Marker(location=[top10_lat[index],top10_lon[index]],icon=folium.Icon(color='green')).add_to(map_Top10)
    folium.Marker(location=[rtop10_lat[index],rtop10_lon[index]],icon=folium.Icon(color='green')).add_to(map_Top10)
    folium.PolyLine(locations=[[top10_lat[index],top10_lon[index]],[rtop10_lat[index],rtop10_lon[index]]], color = 'blue').add_to(map_Top10)
    index = index + 1
map_Top10.save('Top10.html')